# Import Module

In [1]:
import sys
import textwrap
names = sorted(sys.modules.keys())
name_text = ', '.join(names)
print(textwrap.fill(name_text, width=64))

IPython, IPython.core, IPython.core.alias,
IPython.core.application, IPython.core.autocall,
IPython.core.builtin_trap, IPython.core.compilerop,
IPython.core.completer, IPython.core.completerlib,
IPython.core.crashhandler, IPython.core.debugger,
IPython.core.display, IPython.core.display_trap,
IPython.core.displayhook, IPython.core.displaypub,
IPython.core.error, IPython.core.events, IPython.core.excolors,
IPython.core.extensions, IPython.core.formatters,
IPython.core.getipython, IPython.core.history,
IPython.core.hooks, IPython.core.inputsplitter,
IPython.core.inputtransformer, IPython.core.interactiveshell,
IPython.core.latex_symbols, IPython.core.logger,
IPython.core.macro, IPython.core.magic,
IPython.core.magic_arguments, IPython.core.magics,
IPython.core.magics.auto, IPython.core.magics.basic,
IPython.core.magics.code, IPython.core.magics.config,
IPython.core.magics.display, IPython.core.magics.execution,
IPython.core.magics.extension, IPython.core.magics.history,
IPython.core.magi

# Built-in Modules

In [3]:
import sys
import textwrap
name_text = ', '.join(sorted(sys.builtin_module_names))
print(textwrap.fill(name_text, width=64))

_ast, _codecs, _collections, _functools, _imp, _io, _locale,
_operator, _signal, _sre, _stat, _string, _symtable, _thread,
_tracemalloc, _warnings, _weakref, atexit, builtins, errno,
faulthandler, gc, itertools, marshal, posix, pwd, sys, time,
xxsubtype, zipimport


# Import Path

In [4]:
import sys
for d in sys.path:
    print(d)


/Users/gaufung/anaconda/lib/python36.zip
/Users/gaufung/anaconda/lib/python3.6
/Users/gaufung/anaconda/lib/python3.6/lib-dynload
/Users/gaufung/anaconda/lib/python3.6/site-packages
/Users/gaufung/anaconda/lib/python3.6/site-packages/Sphinx-1.5.1-py3.6.egg
/Users/gaufung/anaconda/lib/python3.6/site-packages/aeosa
/Users/gaufung/anaconda/lib/python3.6/site-packages/PyImageQualityRanking-0.1-py3.6.egg
/Users/gaufung/anaconda/lib/python3.6/site-packages/setuptools-27.2.0-py3.6.egg
/Users/gaufung/anaconda/lib/python3.6/site-packages/torchvision-0.1.8-py3.6.egg
/Users/gaufung/anaconda/lib/python3.6/site-packages/IPython/extensions
/Users/gaufung/.ipython


# Importing from a shelve

In [5]:
import shelve
import os

filename = '/tmp/pymotw_import_example.shelve'
if os.path.exists(filename + '.db'):
    os.unlink(filename + '.db')
with shelve.open(filename) as db:
    db['data:README'] = b"""
==============
package README
==============

This is the README for ``package``.
"""
    db['package.__init__'] = b"""
print('package imported')
message = 'This message is in package.__init__'
"""
    db['package.module1'] = b"""
print('package.module1 imported')
message = 'This message is in package.module1'
"""
    db['package.subpackage.__init__'] = b"""
print('package.subpackage imported')
message = 'This message is in package.subpackage.__init__'
"""
    db['package.subpackage.module2'] = b"""
print('package.subpackage.module2 imported')
message = 'This message is in package.subpackage.module2'
"""
    db['package.with_error'] = b"""
print('package.with_error being imported')
raise ValueError('raising exception to break import')
"""
    print('Created {} with:'.format(filename))
    for key in sorted(db.keys()):
        print('  ', key)

Created /tmp/pymotw_import_example.shelve with:
   data:README
   package.__init__
   package.module1
   package.subpackage.__init__
   package.subpackage.module2
   package.with_error


In [6]:
import imp
import os
import shelve
import sys


def _mk_init_name(fullname):
    """Return the name of the __init__ module
    for a given package name.
    """
    if fullname.endswith('.__init__'):
        return fullname
    return fullname + '.__init__'


def _get_key_name(fullname, db):
    """Look in an open shelf for fullname or
    fullname.__init__, return the name found.
    """
    if fullname in db:
        return fullname
    init_name = _mk_init_name(fullname)
    if init_name in db:
        return init_name
    return None


class ShelveFinder:
    """Find modules collected in a shelve archive."""

    _maybe_recursing = False

    def __init__(self, path_entry):
        # Loading shelve causes an import recursive loop when it
        # imports dbm, and we know we are not going to load the
        # module # being imported, so when we seem to be
        # recursing just ignore the request so another finder
        # will be used.
        if ShelveFinder._maybe_recursing:
            raise ImportError
        try:
            # Test the path_entry to see if it is a valid shelf
            try:
                ShelveFinder._maybe_recursing = True
                with shelve.open(path_entry, 'r'):
                    pass
            finally:
                ShelveFinder._maybe_recursing = False
        except Exception as e:
            print('shelf could not import from {}: {}'.format(
                path_entry, e))
            raise
        else:
            print('shelf added to import path:', path_entry)
            self.path_entry = path_entry
        return

    def __str__(self):
        return '<{} for {!r}>'.format(self.__class__.__name__,
                                      self.path_entry)

    def find_module(self, fullname, path=None):
        path = path or self.path_entry
        print('\nlooking for {!r}\n  in {}'.format(
            fullname, path))
        with shelve.open(self.path_entry, 'r') as db:
            key_name = _get_key_name(fullname, db)
            if key_name:
                print('  found it as {}'.format(key_name))
                return ShelveLoader(path)
        print('  not found')
        return None


class ShelveLoader:
    """Load source for modules from shelve databases."""

    def __init__(self, path_entry):
        self.path_entry = path_entry
        return

    def _get_filename(self, fullname):
        # Make up a fake filename that starts with the path entry
        # so pkgutil.get_data() works correctly.
        return os.path.join(self.path_entry, fullname)

    def get_source(self, fullname):
        print('loading source for {!r} from shelf'.format(
            fullname))
        try:
            with shelve.open(self.path_entry, 'r') as db:
                key_name = _get_key_name(fullname, db)
                if key_name:
                    return db[key_name]
                raise ImportError(
                    'could not find source for {}'.format(
                        fullname)
                )
        except Exception as e:
            print('could not load source:', e)
            raise ImportError(str(e))

    def get_code(self, fullname):
        source = self.get_source(fullname)
        print('compiling code for {!r}'.format(fullname))
        return compile(source, self._get_filename(fullname),
                       'exec', dont_inherit=True)

    def get_data(self, path):
        print('looking for data\n  in {}\n  for {!r}'.format(
            self.path_entry, path))
        if not path.startswith(self.path_entry):
            raise IOError
        path = path[len(self.path_entry) + 1:]
        key_name = 'data:' + path
        try:
            with shelve.open(self.path_entry, 'r') as db:
                return db[key_name]
        except Exception:
            # Convert all errors to IOError
            raise IOError()

    def is_package(self, fullname):
        init_name = _mk_init_name(fullname)
        with shelve.open(self.path_entry, 'r') as db:
            return init_name in db

    def load_module(self, fullname):
        source = self.get_source(fullname)

        if fullname in sys.modules:
            print('reusing module from import of {!r}'.format(
                fullname))
            mod = sys.modules[fullname]
        else:
            print('creating a new module object for {!r}'.format(
                fullname))
            mod = sys.modules.setdefault(
                fullname,
                imp.new_module(fullname)
            )

        # Set a few properties required by PEP 302
        mod.__file__ = self._get_filename(fullname)
        mod.__name__ = fullname
        mod.__path__ = self.path_entry
        mod.__loader__ = self
        # PEP-366 specifies that package's set __package__ to
        # their name, and modules have it set to their parent
        # package (if any).
        if self.is_package(fullname):
            mod.__package__ = fullname
        else:
            mod.__package__ = '.'.join(fullname.split('.')[:-1])

        if self.is_package(fullname):
            print('adding path for package')
            # Set __path__ for packages
            # so we can find the sub-modules.
            mod.__path__ = [self.path_entry]
        else:
            print('imported as regular module')

        print('execing source...')
        exec(source, mod.__dict__)
        print('done')
        return mod

In [8]:
import sys



def show_module_details(module):
    print('  message    :', module.message)
    print('  __name__   :', module.__name__)
    print('  __package__:', module.__package__)
    print('  __file__   :', module.__file__)
    print('  __path__   :', module.__path__)
    print('  __loader__ :', module.__loader__)


filename = '/tmp/pymotw_import_example.shelve'
sys.path_hooks.append(sys_shelve_importer.ShelveFinder)
sys.path.insert(0, filename)

print('Import of "package":')
import package

print()
print('Examine package details:')
show_module_details(package)

print()
print('Global settings:')
print('sys.modules entry:')
print(sys.modules['package'])

NameError: name 'sys_shelve_importer' is not defined

# Importer Cache

In [9]:
import os
import sys

prefix = os.path.abspath(sys.prefix)

print('PATH:')
for name in sys.path:
    name = name.replace(prefix, '...')
    print(' ', name)

print()
print('IMPORTERS:')
for name, cache_value in sys.path_importer_cache.items():
    if '..' in name:
        name = os.path.abspath(name)
    name = name.replace(prefix, '...')
    print('  {}: {!r}'.format(name, cache_value))

PATH:
  
  .../lib/python36.zip
  .../lib/python3.6
  .../lib/python3.6/lib-dynload
  .../lib/python3.6/site-packages
  .../lib/python3.6/site-packages/Sphinx-1.5.1-py3.6.egg
  .../lib/python3.6/site-packages/aeosa
  .../lib/python3.6/site-packages/PyImageQualityRanking-0.1-py3.6.egg
  .../lib/python3.6/site-packages/setuptools-27.2.0-py3.6.egg
  .../lib/python3.6/site-packages/torchvision-0.1.8-py3.6.egg
  .../lib/python3.6/site-packages/IPython/extensions
  /Users/gaufung/.ipython

IMPORTERS:
  .../lib/python36.zip: None
  .../lib/python3.6: FileFinder('/Users/gaufung/anaconda/lib/python3.6')
  .../lib/python3.6/encodings: FileFinder('/Users/gaufung/anaconda/lib/python3.6/encodings')
  .../lib/python3.6/lib-dynload: FileFinder('/Users/gaufung/anaconda/lib/python3.6/lib-dynload')
  .../lib/python3.6/collections: FileFinder('/Users/gaufung/anaconda/lib/python3.6/collections')
  .../lib/python3.6/site-packages: FileFinder('/Users/gaufung/anaconda/lib/python3.6/site-packages')
  .../lib/